<a href="https://colab.research.google.com/github/catalinamagalvan/PySimMIBCI/blob/main/FBCNet%20Toolbox/Example_cross_session_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cross-session classification without and with data augmentation
This Google colab notebook illustrates how to employ user-specific simulated data for data augmentation in a cross-session scenario (train: session 1, test: session 2). In this example, real data consist of data from subject 27 of OpenBMI/korea dataset (Dataset 2) and simulated data comprise 100 simulated trials. FBCNet was used as decoding model and only one random seed is considered. The model is trained and tested without and with data augmentation.
 

Notes:

*   Here, the simulated data was previously generated and is directly downloaded in order to reduce execution times. For more information on how to generate this user-specific MI-EEG data go to the example ["Example_generate_data_for_augmentation.ipynb"](https://github.com/catalinamagalvan/PySimMIBCI/blob/main/PySimMIBCI%20Toolbox/Example_generate_data_for_augmentation.ipynb).
*   This notebook has been run by using data from subject 27 of Dataset-2. Note that Figure 7 of the manuscript shows the average of these results over 10 random seeds for each subject of Dataset-2.
*   FBCNet codes are a modification of the [original implementation](https://https://github.com/ravikiran-mane/FBCNet).

##Clone the GitHub repository

In [1]:
!git clone https://github.com/catalinamagalvan/PySimMIBCI.git

Cloning into 'PySimMIBCI'...
remote: Enumerating objects: 273, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 273 (delta 79), reused 109 (delta 41), pack-reused 86
Receiving objects: 100% (273/273), 1.16 MiB | 1.79 MiB/s, done.
Resolving deltas: 100% (121/121), done.


##Install and import required Python libraries

In [2]:
!pip install numpy==1.22.3
!pip install scipy==1.8.1
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 28.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.6/1.6 GB 186.6 MB/s eta 0:00:01tcmalloc: large alloc 1636999168 bytes == 0x20ba000 @  0x7f4554789680 0x7f45547aa824 0x5b3128 0x5bbc90 0x5f714c 0x64d800 0x527022 0x504866 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x5f5ee6 0x56bbe1 0x569d8a 0x5f60c3 0x56cc92 0x569d8a 0x5f60c3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.6/1.6 GB 147.9 MB/s eta 0:00:01tcmalloc: large alloc 2046255

In [3]:
import sys
import os
sys.path.insert(1, os.path.join('PySimMIBCI', 'FBCNet Toolbox', 'codes',
                                'centralRepo'))
from saveData import fetchData
from classification_scenarios import cross_session, cross_session_aug

##Download and prepare real and simulated data

This step must be done in order to make sure that all needed data is present and to generate files with suitable format for FBCNet training.


In [4]:
if not os.path.exists(os.path.join('PySimMIBCI', 'FBCNet Toolbox', 'data',
                                   'koreasim', 'originalData')):
  os.makedirs(os.path.join('PySimMIBCI', 'FBCNet Toolbox', 'data', 'koreasim',
                           'originalData'))
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1cMnImYuxHsNTUYeCj1iw9PH_AA7RSehK' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1cMnImYuxHsNTUYeCj1iw9PH_AA7RSehK" -O PySimMIBCI/FBCNet\ Toolbox/data/koreasim/originalData/ss_27.mat && rm -rf /tmp/cookies.txt

--2023-01-23 16:41:48--  https://docs.google.com/uc?export=download&confirm=t&id=1cMnImYuxHsNTUYeCj1iw9PH_AA7RSehK
Resolving docs.google.com (docs.google.com)... 74.125.24.102, 74.125.24.101, 74.125.24.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.24.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-6o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/41e5jvmi0b3cgbvgtqscc2bd3ibeie83/1674492075000/07130511430595189058/*/1cMnImYuxHsNTUYeCj1iw9PH_AA7RSehK?e=download&uuid=47b02caf-e088-4393-a50a-ea597d4be897 [following]
--2023-01-23 16:41:48--  https://doc-0s-6o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/41e5jvmi0b3cgbvgtqscc2bd3ibeie83/1674492075000/07130511430595189058/*/1cMnImYuxHsNTUYeCj1iw9PH_AA7RSehK?e=download&uuid=47b02caf-e088-4393-a50a-ea597d4be897
Resolving doc-0s-6o-docs.googleusercontent.com (doc-0s-6o-docs.googleusercontent.com)... 142.251.10.132, 2404

In [5]:
fetchData('korea', subjects=[27])
fetchData('koreasim', subjects=[27])

Checking data for dataset:  korea
Extracting the data into mat format: 
Processed data be saved in folder : /content/PySimMIBCI/FBCNet Toolbox/data/korea/rawMat
Processing subject: s27
Fetching data over ftp.
Processing subject: se27
Fetching data over ftp.
Converting to eegdataset.
Creating python eegdataset with raw data.
Converting to multi-view eegdataset.
Outputs will be saved in folder : /content/PySimMIBCI/FBCNet Toolbox/data/korea/multiviewPython
All the data you need is present! 
Checking data for dataset:  koreasim
Extracting the data into mat format: 
Processed data be saved in folder : /content/PySimMIBCI/FBCNet Toolbox/data/koreasim/rawMat
Processing subject: ss_27
Converting to eegdataset.
Creating python eegdataset with raw data.
Converting to multi-view eegdataset.
Outputs will be saved in folder : /content/PySimMIBCI/FBCNet Toolbox/data/koreasim/multiviewPython
All the data you need is present! 


##Train and test the model without data augmentation (only real MI-EEG data)

In [6]:
cross_session(dataset='korea', network='FBCNet', random_seed_list=[0])


Outputs will be saved in folder : /content/PySimMIBCI/FBCNet Toolbox/output/cross_session/FBCNet/korea/seed0
Data loading finished
Trainable Parameters in the network are: 8930
Code will be running on device  cuda:0
Results will be saved in folder : /content/PySimMIBCI/FBCNet Toolbox/output/cross_session/FBCNet/korea/seed0/sub028
	 	 Epoch 1
Train loss = 0.454 Train Acc = 0.825 Val Acc = 0.750 Val loss = 0.467
	 	 Epoch 2
Train loss = 0.305 Train Acc = 0.906 Val Acc = 0.825 Val loss = 0.351
	 	 Epoch 3
Train loss = 0.210 Train Acc = 0.950 Val Acc = 0.900 Val loss = 0.273
	 	 Epoch 4
Train loss = 0.154 Train Acc = 0.975 Val Acc = 0.900 Val loss = 0.231
	 	 Epoch 5
Train loss = 0.123 Train Acc = 0.969 Val Acc = 0.975 Val loss = 0.189
	 	 Epoch 6
Train loss = 0.105 Train Acc = 0.981 Val Acc = 0.950 Val loss = 0.192
	 	 Epoch 7
Train loss = 0.081 Train Acc = 0.994 Val Acc = 0.950 Val loss = 0.171
	 	 Epoch 8
Train loss = 0.075 Train Acc = 1.000 Val Acc = 0.950 Val loss = 0.171
	 	 Epoch 9


##Train and test the model with data augmentation

In [7]:
cross_session_aug(dataset='korea', augmentation='sim', nTrialsAugList=[1],
                  network='FBCNet', random_seed_list=[0])

Outputs will be saved in folder : /content/PySimMIBCI/FBCNet Toolbox/output/cross_session/FBCNet/koreaaug/sim/seed0
Data loading in progress
Data loading finished
Trainable Parameters in the network are: 8930
[61, 92, 174, 22, 79, 41, 112, 132, 59, 111, 95, 81, 17, 80, 53, 90, 83, 154, 96, 148, 161, 38, 39, 130, 6, 93, 58, 3, 194, 0, 133, 120, 135, 26, 186, 178, 121, 166, 139, 77]
Code will be running on device  cuda:0
Results will be saved in folder : /content/PySimMIBCI/FBCNet Toolbox/output/cross_session/FBCNet/koreaaug/sim/seed0/sub028/aug1
	 	 Epoch 1
Train loss = 0.572 Train Acc = 0.688 Val Acc = 0.625 Val loss = 0.597
	 	 Epoch 2
Train loss = 0.387 Train Acc = 0.850 Val Acc = 0.825 Val loss = 0.434
	 	 Epoch 3
Train loss = 0.243 Train Acc = 0.931 Val Acc = 0.925 Val loss = 0.299
	 	 Epoch 4
Train loss = 0.205 Train Acc = 0.956 Val Acc = 0.900 Val loss = 0.285
	 	 Epoch 5
Train loss = 0.155 Train Acc = 0.981 Val Acc = 0.950 Val loss = 0.231
	 	 Epoch 6
Train loss = 0.137 Train Ac